In [1]:
import torch, os, platform, sys
print("PyTorch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
!nvidia-smi

PyTorch: 2.8.0+cu126
CUDA available: True
Thu Nov 13 00:53:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8             11W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!cd /content

!cp /content/drive/MyDrive/ColabNotebooks/beta_decoder_project/mosei_features_and_index.zip .
!unzip -q mosei_features_and_index.zip -d .

In [28]:
!rm -rf /content/HRI-EMO

%cd /content
!git clone https://github.com/Makiato1999/HRI-EMO.git
%cd HRI-EMO

!pwd
!ls

/content
Cloning into 'HRI-EMO'...
remote: Enumerating objects: 298, done.
remote: Counting objects: 100% (298/298), done.
remote: Compressing objects: 100% (221/221), done.
remote: Total 298 (delta 166), reused 199 (delta 71), pack-reused 0 (from 0)
Receiving objects: 100% (298/298), 170.73 KiB | 6.32 MiB/s, done.
Resolving deltas: 100% (166/166), done.
/content/HRI-EMO
/content/HRI-EMO
models	notebooks  README.md  scripts  tests  tools


In [29]:
%cd /content/HRI-EMO

!PYTHONPATH=. python scripts/infer/mosei_eval_infer.py \
  --ckpt /content/drive/MyDrive/ColabNotebooks/beta_decoder_project/HRI-EMO-results/mosei_fusion_decoder_small/best_mosei_fusion_decoder.pt \
  --index_csv_val ../data/mosei_index_splits.csv \
  --index_csv_test ../data/mosei_index_splits.csv \
  --features_root ../features/mosei/seq_level \
  --batch_size 8 \
  --device cuda \
  --max_len_audio 300 \
  --max_len_text 128 \
  --amp_dtype fp16 \
  --beta_hidden 128 \
  --dump_beta \
  --dump_attn \
  --out_dir /content/drive/MyDrive/ColabNotebooks/beta_decoder_project/HRI-EMO-results/mosei_fusion_decoder_small/infer_outputs



/content/HRI-EMO
[✓] Loaded hidden dims from meta.json -> audio=74, text=300
[Saved] val -> /content/drive/MyDrive/ColabNotebooks/beta_decoder_project/HRI-EMO-results/mosei_fusion_decoder_small/infer_outputs/val_y_prob.npy, /content/drive/MyDrive/ColabNotebooks/beta_decoder_project/HRI-EMO-results/mosei_fusion_decoder_small/infer_outputs/val_y_true.npy
[Saved] val beta -> /content/drive/MyDrive/ColabNotebooks/beta_decoder_project/HRI-EMO-results/mosei_fusion_decoder_small/infer_outputs/val_beta_mean.npy
[Saved] test -> /content/drive/MyDrive/ColabNotebooks/beta_decoder_project/HRI-EMO-results/mosei_fusion_decoder_small/infer_outputs/test_y_prob.npy, /content/drive/MyDrive/ColabNotebooks/beta_decoder_project/HRI-EMO-results/mosei_fusion_decoder_small/infer_outputs/test_y_true.npy
[Saved] test beta -> /content/drive/MyDrive/ColabNotebooks/beta_decoder_project/HRI-EMO-results/mosei_fusion_decoder_small/infer_outputs/test_beta_mean.npy


In [30]:
import numpy as np
from sklearn.metrics import f1_score, roc_auc_score

EMO = ["emo_happy","emo_sad","emo_anger","emo_fear","emo_disgust","emo_surprise"]

probs = np.load("/content/drive/MyDrive/ColabNotebooks/beta_decoder_project/HRI-EMO-results/mosei_fusion_decoder_small/infer_outputs/test_y_prob.npy")
y_true_cont = np.load("/content/drive/MyDrive/ColabNotebooks/beta_decoder_project/HRI-EMO-results/mosei_fusion_decoder_small/infer_outputs/test_y_true.npy")
y_true = (y_true_cont > 0).astype(int)

# 阈值：优先用ckpt里保存的（若训练时 --save_calibrated_ths）
import torch
ckpt = torch.load("/content/drive/MyDrive/ColabNotebooks/beta_decoder_project/HRI-EMO-results/mosei_fusion_decoder_small/best_mosei_fusion_decoder.pt", map_location="cpu")
ths = np.array(ckpt.get("val_calibrated_thresholds", [0.5]*len(EMO)), dtype=float)

y_pred = (probs >= ths[None,:]).astype(int)

print("micro-F1:", f1_score(y_true, y_pred, average="micro", zero_division=0))
print("macro-F1:", f1_score(y_true, y_pred, average="macro", zero_division=0))

# macro-ROC-AUC（只对既有正也有负的类统计）
aucs=[]
for i in range(len(EMO)):
    col = y_true[:,i]
    if col.max()>0 and col.min()<1:
        aucs.append(roc_auc_score(col, probs[:,i]))
print("macro-AUC:", float(np.mean(aucs)) if aucs else 0.0)


micro-F1: 0.5053836718004634
macro-F1: 0.39709093193315104
macro-AUC: 0.6687051851286245


In [10]:
import pandas as pd
per_cls = []
for i, name in enumerate(EMO):
    y = y_true[:,i]; p = probs[:,i]
    yhat = (p>=ths[i]).astype(int)
    per_cls.append({"emotion":name,
                    "threshold":ths[i],
                    "f1": f1_score(y,yhat,zero_division=0)})
pd.DataFrame(per_cls).to_csv("/content/drive/MyDrive/ColabNotebooks/beta_decoder_project/HRI-EMO-results/mosei_fusion_decoder_small/infer_outputs/test_per_class_f1.csv", index=False)


In [11]:
from sklearn.metrics import precision_recall_curve, average_precision_score
i = 0  # 选某一类
prec, rec, _ = precision_recall_curve(y_true[:,i], probs[:,i])
ap = average_precision_score(y_true[:,i], probs[:,i])
print(EMO[i], "AP=", ap)


emo_happy AP= 0.73327604450713


In [31]:
!PYTHONPATH=. python scripts/infer/summary_metrics.py

[✓] Loaded thresholds from ckpt (6)

=== Overall Metrics ===
micro-F1 : 0.5054
macro-F1 : 0.3971
macro-AUC: 0.6687

[Saved] per-class results -> summary_metrics.csv
        emotion  threshold      f1      AP
0     emo_happy       0.05  0.7121  0.7333
1       emo_sad       0.10  0.4447  0.3861
2     emo_anger       0.15  0.3785  0.3512
3      emo_fear       0.15  0.2125  0.1594
4   emo_disgust       0.25  0.4824  0.4539
5  emo_surprise       0.15  0.1523  0.1055


In [15]:
!PYTHONPATH=. python scripts/infer/plot_metrics.py --split test --plot_roc

[✓] Figures saved to /content/drive/MyDrive/ColabNotebooks/beta_decoder_project/HRI-EMO-results/mosei_fusion_decoder_small/plots
